# Homework 4 (Propositional Logic)

In [2]:
!pip install sympy==1.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 27.6 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cud

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [1]:
import numpy as np
import sympy

## Propositional Sentence Evaluation

### Utilities


**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

import collections

## Common logic data structures
Not = collections.namedtuple("Not", ["sentence"])
And = collections.namedtuple("And", ["sentence1", "sentence2"])
Or = collections.namedtuple("Or", ["sentence1", "sentence2"])
Implies = collections.namedtuple("Implies", ["sentence1", "sentence2"])

## Propositional logic data structures
Proposition = str  # Name of the proposition
PropositionalModel = dict  # Proposition -> bool

## Example of PropositionalModel, used in tests
IS_RAINING = Proposition("is-raining")
IS_SUNNY = Proposition("is-sunny")
NEED_UMBRELLA = Proposition("need-umbrella")
PROP_MODEL = PropositionalModel({
    IS_RAINING: True,
    IS_SUNNY: False,
    NEED_UMBRELLA: True,
})

### Question
*Note: for these questions, refer to the top of the Colab notebook.*
Write a function that takes a propositional sentence and evaluates it against a single model.
You may find python's builtin `isinstance` useful. For example, `isinstance(sentence, And)` returns whether a sentence is an `And`.

For reference, our solution is **15** line(s) of code.

In [ ]:
def evaluate_propositional_sentence(sentence, model):
    """Evaluate a propositional sentence against a single model.

    Args:
      sentence: A Proposition, And, Or, Not, or Implies.
      model: A PropositionalModel.

    Returns:
      holds: A bool representing the truth value of the sentence
        under the model.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
assert evaluate_propositional_sentence(IS_RAINING, PROP_MODEL) == True


assert evaluate_propositional_sentence(Not(IS_RAINING), PROP_MODEL) == False


assert evaluate_propositional_sentence(And(IS_RAINING, IS_SUNNY),
                                       PROP_MODEL) == False


assert evaluate_propositional_sentence(Implies(IS_SUNNY, Not(IS_RAINING)),
                                       PROP_MODEL) == True


assert evaluate_propositional_sentence(Or(IS_RAINING, Not(IS_RAINING)),
                                       PROP_MODEL) == True

assert evaluate_propositional_sentence(And(Or(Not(IS_RAINING), Not(Not(NEED_UMBRELLA))),
                                           Or(IS_SUNNY, Not(Not(Implies(Not(IS_RAINING), IS_SUNNY))))),
                                       PROP_MODEL) == True

print('Tests passed.')

## Warmup


### Utilities


**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
def lit_to_var_val(literal):
    """Converts a literal into (variable, value).

    Args:
      literal: A nonzero int.

    Returns:
      variable: A positive int representing a variable.
      value: True or False, i.e., positive or negative.
    """
    return abs(literal), literal > 0


def is_cnf_formula(formula):
    """Checks whether the input is a valid CNF formula.

    A formula is in valid CNF form if it is a list of lists of nonzero
    integers with sign indicating whether the variable is negated.

    You will not need to use this utility in your implementation,
    but it may be useful to read to understand the CNF representation.
    """
    if not isinstance(formula, list):
        return False
    if len(formula) == 0:
        return True
    clause = formula[0]
    if not isinstance(clause, list):
        return False
    for literal in clause:
        if not isinstance(literal, int):
            return False
        if literal == 0:
            return False
    if len(formula) == 1:
        return True
    return is_cnf_formula(formula[1:])


def get_variables_in_cnf_formula(cnf_formula):
    """Get a list of all variables in a CNF formula.

    Args:
      cnf_formula: A list of lists of nonzero ints.

    Returns:
      variables: A list of all variables that appear in
        the formula.
    """
    variables = set()
    for clause in cnf_formula:
        variables.update({lit_to_var_val(literal)[0] for literal in clause})
    variables = sorted(variables)
    return variables

### Question
In this problem, CNF formulas are represented as lists of lists of nonzero integers. The sign of the integer represents whether the corresponding proposition is negated or not. For example, the formula ((x1 or not x2) and (x3 or x2)) would be represented as [[1, -2], [3, 2]]. Complete the following function to confirm your understanding of this representation.

For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup():
    """Return a list of lists of ints for the CNF formula:

    ((x4 or not x5 or not x6) and (x6 or x5 or not x1) and (x2 or x3)).

    Keep the same order as in the formula above.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
# Note: the catsoop test is more strict for this question
# than the unit tests here. Make sure that your answer
# matches the description in the docstring exactly.
assert is_cnf_formula(warmup())
assert get_variables_in_cnf_formula(warmup()) == [1, 2, 3, 4, 5, 6]

print('Tests passed.')

## [OPTIONAL] DPLL


### Utilities

**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:


def clause_is_determined(clause, partial_assignment):
    """Checks whether all variables in the clause have an assignment.

    Args:
      clause: A list of nonzero ints.
      partial_assignment: A dict of variables (ints) to values (bools).

    Returns:
      is_determined: True if all variables in the clause appear in
        partial_assignment.
    """
    for literal in clause:
        if not (literal in partial_assignment or
                -literal in partial_assignment):
            return False
    return True


def literal_is_satisfied(literal, partial_assignment):
    """Checks whether the literal is satisfied by the assignment.

    Args:
      literal: A nonzero int.
      partial_assignment: A dict of variables (ints) to values (bools).

    Returns:
      is_satisfied: True if the literal's variable appears in the
        partial_assignment, with a sign matching the literal.
    """
    variable, val = lit_to_var_val(literal)
    return variable in partial_assignment and partial_assignment[variable] == val


def clause_is_satisfied(clause, partial_assignment):
    """Checks whether the clause is satisfied by the assignment.

    Args:
      clause: A list of nonzero ints.
      partial_assignment: A dict of variables (ints) to values (bools).

    Returns:
      is_satisfied: True if some literal in the clause is satisfied.
    """
    for literal in clause:
        if literal_is_satisfied(literal, partial_assignment):
            return True
    return False


def find_pure_variable(cnf_formula, variables, partial_assignment):
    """Helper for DPLL.

    A variable is pure if it has the same sign in all unsatisfied clauses
    and if it is not already assigned.

    If a pure variable exists, this function returns the variable and value
    corresponding to the literal. (If multiple exist, return an arbitrary one.)

    If no pure variables exist, return (None, None).

    Args:
      cnf_formula: A list of lists of nonzero integers representing a CNF formula,
        with sign indicating whether the variable is negated.
      variables: A list of positive integers.
      partial_assignment: A dict mapping positive integers to bools, or None for
        an empty assignment.

    Returns:
      variable : A positive integer or None.
      value: A bool or None.
    """
    candidate_to_possible_values  = {v : {True, False} for v in variables \
                                    if v not in partial_assignment}
    for clause in cnf_formula:
        if clause_is_satisfied(clause, partial_assignment):
            continue
        for literal in clause:
            variable, value = lit_to_var_val(literal)
            if variable in candidate_to_possible_values:
                candidate_to_possible_values[variable].discard(not value)
    for candidate, possible_values in candidate_to_possible_values.items():
        if possible_values:
            value = next(iter(possible_values))
            return candidate, value
    return None, None


def find_unit_clause(cnf_formula, partial_assignment):
    """Helper for DPLL.

    A clause is a unit clause if all literals but one are already assigned to
    False. If a unit clause exists, this function returns the variable and value
    corresponding to the literal. (If multiple exist, return an arbitrary one.)

    Args:
      cnf_formula: A list of lists of nonzero integers representing a CNF formula,
        with sign indicating whether the variable is negated.
      variables: A list of positive integers.
      partial_assignment: A dict mapping positive integers to bools, or None for
        an empty assignment.

    Returns:
      variable : A positive integer or None.
      value: A bool or None.
    """
    for clause in cnf_formula:
        unassigned_literal = None
        is_unit_clause = True
        for literal in clause:
            # If the literal is true in the assignment, this is not a unit clause
            if literal_is_satisfied(literal, partial_assignment):
                is_unit_clause = False
                break
            # If the literal is false in the assignment, this could be a unit clause
            elif literal in partial_assignment:
                continue
            # If there is already an unassigned literal, this is not a unit clause
            elif not (unassigned_literal is None):
                is_unit_clause = False
                break
            else:
                unassigned_literal = literal
        if is_unit_clause and not (unassigned_literal is None):
            return lit_to_var_val(unassigned_literal)
    return None, None

### [OPTIONAL] Question

**This question is optional.** Uncomment the tests below if you'd like to work on it. You can find an explanation of it and pseudocode in AIMA section 7.6.

Use your helper functions to complete an implementation of DPLL.

For reference, our solution is **59** line(s) of code.

In [ ]:
def run_inference_dpll(cnf_formula):
    """Find a satisfying assignment for a propositional CNF formula with DPLL.

    Args:
      cnf_formula: A list of lists of nonzero integers representing a CNF formula,
        with sign indicating whether the variable is negated.

    Returns:
      satisfiable: A bool indicating whether some satisfying assignment exists.
      assignment: A dict mapping positive integers to bools, or None if no
        satisfying assignment exists.

    Examples:
      >> run_inference_dpll([[1, -2], [-1, -2]])
      >> (True, {1: True, 2: False}))

      >> run_inference_dpll([[1], [-1]])
      >> (False, None)
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
""" Uncomment to work on this problem!
assert run_inference_dpll([]) == (True, {})


assert run_inference_dpll([[1]]) == (True, {1: True})


assert run_inference_dpll([[-1]]) == (True, {1: False})


assert run_inference_dpll([[1, 2]]) in [(True, {1: True, 2: True}), (True, {1: True, 2: False}), (True, {1: False, 2: True})]


assert run_inference_dpll([[-1, 2]]) in [(True, {1: True, 2: True}), (True, {1: False, 2: True}), (True, {1: False, 2: False})]


assert run_inference_dpll([[1], [-1]]) == (False, None)


assert run_inference_dpll([[1, 2, 3], [-1, -2, -3], [1, -2, 3], [-1], [-3]]) == (False, None)


assert run_inference_dpll([[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]) == (True, {1: True, 2: True, 3: True, 4: True, 5: True, 6: True, 7: True, 8: True, 9: True, 10: True, 11: True, 12: True, 13: True, 14: True, 15: True, 16: True, 17: True, 18: True, 19: True, 20: True, 21: True, 22: True, 23: True, 24: True, 25: True, 26: True, 27: True, 28: True, 29: True, 30: True, 31: True, 32: True})
print('Tests passed.')
"""

## Sympy Warmup 1


### Question
Use sympy to determine whether the following formula is satisfiable:
$(\neg x_1 \land x_2) \Rightarrow ((x_2 \lor x_3) \land (x_1 \lor \neg x_3))$.

Note that the return type should be **bool**.


For reference, our solution is **4** line(s) of code.

In [ ]:
def formula1_is_satisfiable():
    """Determines whether the above formula is satisfiable.

    Returns:
      is_satisfiable: A bool indicating whether the formula is satisfiable.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:

assert formula1_is_satisfiable() == True
print('Tests passed.')

## Sympy Warmup 2


### Question
Use sympy to determine whether the following formula is satisfiable:
$(x_1 \lor x_2) \land (\neg x_1 \lor \neg x_2) \land (x_1 \lor \neg x_2) \land (\neg x_1 \lor x_2)$.


For reference, our solution is **4** line(s) of code.

In [ ]:
def formula2_is_satisfiable():
    """Determines whether the above formula is satisfiable.

    Returns:
      is_satisfiable: A bool indicating whether the formula is satisfiable.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:

assert formula2_is_satisfiable() == False
print('Tests passed.')

## Search and Rescue Inference

### Question
Write a program that takes a grid as input and infers unknown values.

Your program should output a new grid with all determinable unknown values replaced with the inferred value. If an unknown value cannot be determined, it should be left unknown.

**Your program should use sympy.**


For reference, our solution is **57** line(s) of code.

In [37]:
def get_neighbors(grid, row, col):
  neighbors = []
  for x,y in [[-1,0], [1,0], [0,-1], [0,1]]:
    if 0 <= row+x < len(grid) and 0 <= col+y < len(grid[0]):
      neighbors.append((row+x,col+y))
  return neighbors

def infer_unknown_values(grid):
    """Fill in any unknown values in the grid that can be inferred.

    Args:
      grid: A list of lists of "F", "U", "S", or "C".

    Returns:
      inferred_grid: A copy of grid with some unknown values replaced.

    Example:
      >> grid = [
      >>   ["F", "U", "C"],
      >>   ["S", "C", "U"],
      >>   ["U", "U", "C"]
      >> ]
      >> infer_unknown_values(grid)
      >> [["F" "S" "C"]
      >>  ["S" "C" "C"]
      >>  ["U" "U" "C"]]
    """
    formula = True

    for row in range(len(grid)):
      for col in range(len(grid[0])):

        neighbors = get_neighbors(grid, row, col)

        c, s, f, u = sympy.symbols(f"{row}_{col}_C {row}_{col}_S {row}_{col}_F {row}_{col}_U")

        fire = False
        determined = True
        smoke = True
        clear = True
        # maybe_fire = False

        for r, cl in neighbors:
          n_c, n_s, n_f, n_u = sympy.symbols(f"{r}_{cl}_C {r}_{cl}_S {r}_{cl}_F {r}_{cl}_U")
          fire = fire | n_f
          clear = clear & n_c
          smoke = smoke & n_s
          determined = determined & ~n_u
          # maybe_fire = maybe_fire & (n_f | n_u)


        # smoke here if and only if fire around
        formula = formula & sympy.Implies(s, fire)
        formula = formula & sympy.Implies(fire, s)
        formula = formula & sympy.Implies(f, smoke)

        # how to add clause "none of your neighbors *can* be fire"
        # formula = formula & sympy.Implies(~smoke & ~maybe_fire, c)
        formula = formula & sympy.Implies(determined & ~fire & ~smoke, c)
        formula = formula & sympy.Implies(clear, c)


        # record this cell is clear
        if grid[row][col] == "C":
          c, s, f, u = sympy.symbols(f"{row}_{col}_C {row}_{col}_S {row}_{col}_F {row}_{col}_U")
          formula = formula & (c & ~s & ~f & ~u)

        # record this cell has smoke
        elif grid[row][col] == "S":
          c, s, f, u = sympy.symbols(f"{row}_{col}_C {row}_{col}_S {row}_{col}_F {row}_{col}_U")
          formula = formula & (~c & s & ~f & ~u)

        # record this cell has fire
        elif grid[row][col] == "F":
          c, s, f, u = sympy.symbols(f"{row}_{col}_C {row}_{col}_S {row}_{col}_F {row}_{col}_U")
          formula = formula & (~c & ~s & f & ~u)

        elif grid[row][col] == "U":
          formula = formula & ((c & ~s & ~f & ~u) | (~c & s & ~f & ~u) | (~c & ~s & f & ~u) | (~c & ~s & ~f & u))
        # print(formula)

    print(formula)
    ans = sympy.satisfiable(formula)
    if not ans:
      return grid
    new_grid = [[0 for _ in range(len(grid[0]))] for _ in range(len(grid))]
    for row in range(len(grid)):
      for col in range(len(grid[0])):
        for letter in ["F", "S", "C", "U"]:
          if ans[sympy.symbols(f"{row}_{col}_{letter}")]:
            new_grid[row][col] = letter
            break
    print(new_grid)
    return new_grid


### Tests

In [38]:

assert infer_unknown_values([["U", "F"]]) == [["S", "F"]]


assert infer_unknown_values([["F", "U", "C"], ["S", "C", "U"], ["U", "U", "C"]]) == [["F", "S", "C"], ["S", "C", "C"], ["U", "U", "C"]]


assert infer_unknown_values([["U", "C", "C"], ["S", "C", "U"], ["U", "U", "C"]]) == [["C", "C", "C"], ["S", "C", "C"], ["F", "S", "C"]]


assert infer_unknown_values([["U", "S", "C", "U"], ["U", "U", "C", "U"], ["U", "S", "C", "U"]]) == [["F", "S", "C", "C"], ["S", "C", "C", "C"], ["F", "S", "C", "C"]]


assert infer_unknown_values([["U", "U", "C", "U", "U", "U", "U", "U"], ["C", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "C", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "F", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"]]) == [["C", "C", "C", "U", "U", "U", "U", "U"], ["C", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "C", "U", "S", "U", "U", "U", "U"], ["U", "U", "S", "F", "S", "U", "U", "U"], ["U", "U", "U", "S", "U", "U", "U", "U"]]
print('Tests passed.')

0_1_F & ~0_1_C & ~0_1_S & ~0_1_U & (Implies(0_0_C, 0_1_C)) & (Implies(0_0_F, 0_1_S)) & (Implies(0_0_S, 0_1_F)) & (Implies(0_1_C, 0_0_C)) & (Implies(0_1_F, 0_0_S)) & (Implies(0_1_S, 0_0_F)) & (Implies(~0_0_F & ~0_0_S & ~0_0_U, 0_1_C)) & (Implies(~0_1_F & ~0_1_S & ~0_1_U, 0_0_C)) & ((0_0_C & ~0_0_F & ~0_0_S & ~0_0_U) | (0_0_F & ~0_0_C & ~0_0_S & ~0_0_U) | (0_0_S & ~0_0_C & ~0_0_F & ~0_0_U) | (0_0_U & ~0_0_C & ~0_0_F & ~0_0_S))
[['S', 'F']]
0_0_F & 0_2_C & 1_0_S & 1_1_C & 2_2_C & ~0_0_C & ~0_0_S & ~0_0_U & ~0_2_F & ~0_2_S & ~0_2_U & ~1_0_C & ~1_0_F & ~1_0_U & ~1_1_F & ~1_1_S & ~1_1_U & ~2_2_F & ~2_2_S & ~2_2_U & (Implies(0_0_F, 0_1_S & 1_0_S)) & (Implies(0_0_S, 0_1_F | 1_0_F)) & (Implies(0_2_F, 0_1_S & 1_2_S)) & (Implies(0_2_S, 0_1_F | 1_2_F)) & (Implies(2_0_F, 1_0_S & 2_1_S)) & (Implies(2_0_S, 1_0_F | 2_1_F)) & (Implies(2_2_F, 1_2_S & 2_1_S)) & (Implies(2_2_S, 1_2_F | 2_1_F)) & (Implies(0_1_C & 1_0_C, 0_0_C)) & (Implies(0_1_C & 1_2_C, 0_2_C)) & (Implies(1_0_C & 2_1_C, 2_0_C)) & (Implies(

AssertionError: 